# Climate Projections Monthly: Global timeseries

## Import libraries

Switch warnings off for better readability.

In [ ]:
import warnings

from c3s_eqc_automatic_quality_control import diagnostics, download, plot

warnings.filterwarnings("ignore")

## Define request

In [ ]:
collection_id = "projections-cmip6"

year_start = 2008
year_stop = 2009

request = {
    "format": "zip",
    "temporal_resolution": "monthly",
    "experiment": "historical",
    "variable": "surface_temperature",
    "model": "cmcc_cm2_sr5",
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 12 + 1)],
}

## Define transform function

Drop bounds as they are not used and create issues with dask

In [ ]:
def drop_bounds_and_spatial_weighted_mean(ds):
    return diagnostics.spatial_weighted_mean(ds.drop_dims("bnds"))


def drop_bounds_and_spatial_weighted_std(ds):
    return diagnostics.spatial_weighted_std(ds.drop_dims("bnds"))

## Compute spatially-weighted global mean timeseries

In [ ]:
ds_mean = download.download_and_transform(
    collection_id,
    request,
    transform_func=drop_bounds_and_spatial_weighted_mean,
    split_all=True,
)

## Compute spatially-weighted global std timeseries

In [ ]:
ds_std = download.download_and_transform(
    collection_id,
    request,
    transform_func=drop_bounds_and_spatial_weighted_std,
    split_all=True,
)

## Plot and save figure

In [ ]:
filename = "_".join(
    ["global-timeseries", collection_id, request["temporal_resolution"]]
)
title = filename.replace("-", " ").replace("_", " ").title()
fig = plot.shaded_std("ts", ds_mean, ds_std, title=title)
fig.show()
fig.write_image(filename + ".png")